# Test Notebook: MaxViT (Multi-Axis Vision Transformer)

Este notebook contiene pruebas básicas para el modelo **MaxViT**, un Vision Transformer que combina atención local y global de manera eficiente.

## Referencias
- Paper: [MaxViT: Multi-Axis Vision Transformer](https://arxiv.org/abs/2204.01697)
- Implementación: PyTorch (torchvision) y timm

## 1. Instalación de dependencias

In [ ]:
# Instalar dependencias necesarias
!pip install torch torchvision timm pillow matplotlib numpy --quiet

## 2. Imports

In [ ]:
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import timm
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform

# Configuración del dispositivo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Usando dispositivo: {device}")

# Verificar versiones
print(f"PyTorch version: {torch.__version__}")
print(f"timm version: {timm.__version__}")

## 3. Explorar modelos MaxViT disponibles

In [ ]:
# Listar todos los modelos MaxViT disponibles en timm
maxvit_models = timm.list_models('*maxvit*', pretrained=True)
print(f"Modelos MaxViT disponibles ({len(maxvit_models)}):")
for model_name in maxvit_models:
    print(f"  - {model_name}")

## 4. Cargar modelo MaxViT preentrenado

In [ ]:
# Cargar MaxViT-Tiny preentrenado en ImageNet (modelo más ligero)
# Opciones disponibles:
#   - maxvit_tiny_tf_224.in1k (más pequeño y rápido)
#   - maxvit_small_tf_224.in1k
#   - maxvit_base_tf_224.in1k (más grande y preciso)

MODEL_NAME = 'maxvit_tiny_tf_224.in1k'

model = timm.create_model(
    MODEL_NAME,
    pretrained=True,
    num_classes=1000  # ImageNet tiene 1000 clases
)
model = model.to(device)
model.eval()

print(f"Modelo cargado: {MODEL_NAME}")
print(f"Número de parámetros: {sum(p.numel() for p in model.parameters()):,}")

## 5. Configurar transformaciones de imagen

In [ ]:
# Obtener la configuración de datos del modelo
data_config = resolve_data_config({}, model=model)
print("Configuración del modelo:")
for key, value in data_config.items():
    print(f"  {key}: {value}")

# Crear transformación para inferencia
transform = create_transform(**data_config, is_training=False)
print(f"\nTransformaciones: {transform}")

## 6. Función de predicción

In [ ]:
def predict_image(image_path, model, transform, top_k=5):
    """
    Realiza una predicción sobre una imagen.
    
    Args:
        image_path: Ruta a la imagen o URL
        model: Modelo de PyTorch
        transform: Transformaciones a aplicar
        top_k: Número de predicciones top a mostrar
    
    Returns:
        Diccionario con las predicciones
    """
    # Cargar imagen
    if image_path.startswith('http'):
        import urllib.request
        from io import BytesIO
        with urllib.request.urlopen(image_path) as url:
            img = Image.open(BytesIO(url.read())).convert('RGB')
    else:
        img = Image.open(image_path).convert('RGB')
    
    # Aplicar transformaciones
    input_tensor = transform(img).unsqueeze(0).to(device)
    
    # Realizar predicción
    with torch.no_grad():
        output = model(input_tensor)
        probabilities = torch.nn.functional.softmax(output[0], dim=0)
    
    # Obtener top-k predicciones
    top_probs, top_indices = torch.topk(probabilities, top_k)
    
    return {
        'image': img,
        'top_k_indices': top_indices.cpu().numpy(),
        'top_k_probs': top_probs.cpu().numpy()
    }

## 7. Cargar etiquetas de ImageNet

In [ ]:
# Descargar etiquetas de ImageNet
import urllib.request
import json

LABELS_URL = "https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt"

try:
    with urllib.request.urlopen(LABELS_URL) as url:
        imagenet_labels = url.read().decode('utf-8').strip().split('\n')
    print(f"Cargadas {len(imagenet_labels)} etiquetas de ImageNet")
    print(f"Ejemplos: {imagenet_labels[:5]}")
except Exception as e:
    print(f"Error cargando etiquetas: {e}")
    imagenet_labels = [f"clase_{i}" for i in range(1000)]

## 8. Test con imagen de ejemplo

In [ ]:
# Probar con una imagen de ejemplo (un gato)
# Puedes cambiar esta URL por cualquier imagen
TEST_IMAGE_URL = "https://upload.wikimedia.org/wikipedia/commons/thumb/3/3a/Cat03.jpg/1200px-Cat03.jpg"

# Realizar predicción
result = predict_image(TEST_IMAGE_URL, model, transform)

# Mostrar resultados
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Mostrar imagen
ax1.imshow(result['image'])
ax1.set_title('Imagen de entrada')
ax1.axis('off')

# Mostrar predicciones
labels = [imagenet_labels[idx] for idx in result['top_k_indices']]
probs = result['top_k_probs'] * 100

colors = plt.cm.Blues(np.linspace(0.4, 0.9, len(labels)))[::-1]
bars = ax2.barh(labels[::-1], probs[::-1], color=colors)
ax2.set_xlabel('Probabilidad (%)')
ax2.set_title('Top-5 Predicciones')
ax2.set_xlim(0, 100)

# Añadir valores en las barras
for bar, prob in zip(bars, probs[::-1]):
    ax2.text(prob + 1, bar.get_y() + bar.get_height()/2, 
             f'{prob:.1f}%', va='center')

plt.tight_layout()
plt.show()

print("\nPredicciones detalladas:")
for idx, (label, prob) in enumerate(zip(labels, probs), 1):
    print(f"  {idx}. {label}: {prob:.2f}%")

## 9. Fine-tuning para clasificación de hojas (ejemplo)

In [ ]:
# Ejemplo de cómo adaptar MaxViT para clasificación de enfermedades de hojas

# Clases del dataset de manzanas
LEAF_CLASSES = [
    'Alternaria leaf spot',
    'Brown spot',
    'Gray spot',
    'Healthy leaf',
    'Rust'
]
NUM_CLASSES = len(LEAF_CLASSES)

def create_finetune_model(model_name='maxvit_tiny_tf_224.in1k', num_classes=5, freeze_backbone=True):
    """
    Crea un modelo MaxViT preparado para fine-tuning.
    
    Args:
        model_name: Nombre del modelo base
        num_classes: Número de clases para la nueva tarea
        freeze_backbone: Si True, congela las capas del backbone
    
    Returns:
        Modelo configurado para fine-tuning
    """
    # Crear modelo con número personalizado de clases
    model = timm.create_model(
        model_name,
        pretrained=True,
        num_classes=num_classes
    )
    
    # Congelar backbone si se especifica
    if freeze_backbone:
        for name, param in model.named_parameters():
            # Solo entrenar la cabeza de clasificación
            if 'head' not in name and 'classifier' not in name:
                param.requires_grad = False
    
    # Contar parámetros entrenables
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total_params = sum(p.numel() for p in model.parameters())
    
    print(f"Modelo: {model_name}")
    print(f"Clases: {num_classes}")
    print(f"Parámetros totales: {total_params:,}")
    print(f"Parámetros entrenables: {trainable_params:,} ({100*trainable_params/total_params:.2f}%)")
    
    return model

# Crear modelo para fine-tuning
finetune_model = create_finetune_model(
    model_name='maxvit_tiny_tf_224.in1k',
    num_classes=NUM_CLASSES,
    freeze_backbone=True
)

## 10. Ejemplo de DataLoader para el dataset de hojas

In [ ]:
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
import os

# Transformaciones para entrenamiento (con aumento de datos)
train_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

# Transformaciones para validación
val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

# # Ejemplo de cómo crear DataLoaders

# DATA_DIR = '../data/manzanas'

# if os.path.exists(DATA_DIR):
#     train_dataset = datasets.ImageFolder(
#         os.path.join(DATA_DIR, 'train'),
#         transform=train_transform
#     )
#     val_dataset = datasets.ImageFolder(
#         os.path.join(DATA_DIR, 'val'),
#         transform=val_transform
#     )
    
#     train_loader = DataLoader(
#         train_dataset,
#         batch_size=32,
#         shuffle=True,
#         num_workers=4
#     )
#     val_loader = DataLoader(
#         val_dataset,
#         batch_size=32,
#         shuffle=False,
#         num_workers=4
#     )
    
#     print(f"Train samples: {len(train_dataset)}")
#     print(f"Val samples: {len(val_dataset)}")
#     print(f"Classes: {train_dataset.classes}")

print("Ejemplo de transformaciones definidas correctamente.")
print(f"Train transform: {train_transform}")

## 11. Bucle de entrenamiento (ejemplo)

In [ ]:
def train_one_epoch(model, train_loader, criterion, optimizer, device):
    """
    Entrena el modelo durante una época.
    """
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        
        if (batch_idx + 1) % 10 == 0:
            print(f'Batch [{batch_idx+1}/{len(train_loader)}] '
                  f'Loss: {running_loss/(batch_idx+1):.4f} '
                  f'Acc: {100.*correct/total:.2f}%')
    
    return running_loss / len(train_loader), 100. * correct / total


def evaluate(model, val_loader, criterion, device):
    """
    Evalúa el modelo en el conjunto de validación.
    """
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for inputs, targets in val_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    
    return running_loss / len(val_loader), 100. * correct / total


# # Ejemplo de configuración de entrenamiento

# model = create_finetune_model(num_classes=5, freeze_backbone=True).to(device)
# criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)
# 
# EPOCHS = 10
# for epoch in range(EPOCHS):
#     print(f"\n=== Epoch {epoch+1}/{EPOCHS} ===")
#     train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device)
#     val_loss, val_acc = evaluate(model, val_loader, criterion, device)
#     scheduler.step()
#     print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
#     print(f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%")

print("Funciones de entrenamiento definidas correctamente.")

## 12. Guardar y cargar modelo

In [ ]:
def save_model(model, path, model_name, num_classes, epoch=None):
    """
    Guarda el modelo con metadatos.
    """
    checkpoint = {
        'model_name': model_name,
        'num_classes': num_classes,
        'state_dict': model.state_dict(),
        'epoch': epoch
    }
    torch.save(checkpoint, path)
    print(f"Modelo guardado en: {path}")


def load_model(path, device='cuda'):
    """
    Carga un modelo guardado.
    """
    checkpoint = torch.load(path, map_location=device)
    
    model = timm.create_model(
        checkpoint['model_name'],
        pretrained=False,
        num_classes=checkpoint['num_classes']
    )
    model.load_state_dict(checkpoint['state_dict'])
    model = model.to(device)
    model.eval()
    
    print(f"Modelo cargado: {checkpoint['model_name']}")
    print(f"Epoch: {checkpoint.get('epoch', 'N/A')}")
    
    return model


# Ejemplo de uso:
# save_model(model, 'maxvit_leaf_classifier.pth', 'maxvit_tiny_tf_224.in1k', NUM_CLASSES, epoch=10)
# model = load_model('maxvit_leaf_classifier.pth', device)

print("Funciones de guardado/carga definidas correctamente.")

## 13. Resumen de arquitectura MaxViT

In [ ]:
# Mostrar resumen de la arquitectura del modelo
print("=" * 60)
print("ARQUITECTURA MAXVIT")
print("=" * 60)
print("""
MaxViT (Multi-Axis Vision Transformer) combina:

1. **Bloques MBConv**: Convoluciones eficientes tipo MobileNetV2
   - Depthwise separable convolutions
   - Squeeze-and-Excitation (SE)

2. **Block Attention**: Atención local dentro de ventanas
   - Divide la imagen en ventanas no superpuestas
   - Aplica self-attention dentro de cada ventana

3. **Grid Attention**: Atención global con patrón de cuadrícula
   - Permite interacción global entre regiones
   - Complejidad lineal respecto al tamaño de imagen

Ventajas:
- ✅ Captura tanto patrones locales como globales
- ✅ Eficiente computacionalmente
- ✅ Funciona bien con diferentes resoluciones
- ✅ Estado del arte en ImageNet y otros benchmarks
""")

# Mostrar estructura del modelo
print("\nEstructura del modelo MaxViT-Tiny:")
print("-" * 40)
for name, module in model.named_children():
    num_params = sum(p.numel() for p in module.parameters())
    print(f"{name}: {num_params:,} parámetros")

---

## Próximos pasos

1. **Preparar el dataset**: Organizar las imágenes en carpetas `train/` y `val/` con subcarpetas por clase
2. **Ajustar hiperparámetros**: Learning rate, batch size, épocas, etc.
3. **Experimentar con diferentes modelos**: Probar `maxvit_small` o `maxvit_base` si se necesita más capacidad
4. **Implementar métricas adicionales**: Precision, Recall, F1-score, matriz de confusión
5. **Técnicas de regularización**: Dropout, weight decay, mixup, cutmix